# Dashboard for F1 Data Insights

### Import Packages

In [1]:
import numpy as np
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import plotly.express as px

# View all rows and columns of a dataframe
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

## Loading in Data

In [2]:
# Load Drivers data
drivers_df = pd.read_csv("./f1db_csv/drivers.csv").drop(columns = "url")
drivers_df

,driverId,driverRef,number,code,forename,surname,dob,nationality
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish
...,...,...,...,...,...,...,...,...
843,845,sirotkin,35,SIR,Sergey,Sirotkin,1995-08-27,Russian
844,846,norris,4,NOR,Lando,Norris,1999-11-13,British
845,847,russell,63,RUS,George,Russell,1998-02-15,British
846,848,albon,23,ALB,Alexander,Albon,1996-03-23,Thai


In [3]:
# Load lap times data
lap_times_df = pd.read_csv("./f1db_csv/lap_times.csv")
lap_times_df = lap_times_df[(lap_times_df.raceId == 1034) | (lap_times_df.raceId == 1033)]
lap_times_df

,raceId,driverId,lap,position,time,milliseconds
474870,1033,1,1,1,1:35.670,95670
474871,1033,1,2,1,1:31.458,91458
474872,1033,1,3,1,1:34.916,94916
474873,1033,1,4,3,1:53.821,113821
474874,1033,1,5,1,1:25.898,85898
...,...,...,...,...,...,...
477087,1034,849,48,15,1:30.871,90871
477088,1034,849,49,15,1:30.915,90915
477089,1034,849,50,15,1:31.119,91119
477090,1034,849,51,15,1:31.227,91227


In [4]:
# Load results data
results_df = pd.read_csv("./f1db_csv/results.csv")
results_df.head(10)
# Filter to only race 1034
# results_1034_df = results_df[results_df.raceId == 1034]

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
5,6,18,6,3,8,13,6,6,6,3.0,57,\N,\N,50,14,1:29.639,212.974,11
6,7,18,7,5,14,17,7,7,7,2.0,55,\N,\N,22,12,1:29.534,213.224,5
7,8,18,8,6,1,15,8,8,8,1.0,53,\N,\N,20,4,1:27.903,217.180,5
8,9,18,9,2,4,2,\N,R,9,0.0,47,\N,\N,15,9,1:28.753,215.100,4
9,10,18,10,7,12,18,\N,R,10,0.0,43,\N,\N,23,13,1:29.558,213.166,3


In [5]:
# Load constructors names
constructors_df = pd.read_csv("./f1db_csv/constructors.csv")
constructors_df

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso
...,...,...,...,...,...
206,209,manor,Manor Marussia,British,http://en.wikipedia.org/wiki/Manor_Motorsport
207,210,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team
208,211,racing_point,Racing Point,British,http://en.wikipedia.org/wiki/Racing_Point_F1_Team
209,212,alpha_tauri,Scuderia Alpha Tauri,Italian,http://en.wikipedia.org/wiki/Scuderia_Alpha_Tauri


In [6]:
# Load race ID and names
races_df = pd.read_csv("./f1db_csv/races.csv")

In [7]:
# Some cleaning
clean_lt_df = lap_times_df[["raceId", "driverId", "lap", "milliseconds"]]
clean_lt_df["seconds"] = clean_lt_df.milliseconds / 1000
clean_lt_df = clean_lt_df.drop(columns = "milliseconds")
clean_lt_df

# race_1034_df = clean_lt_df[clean_lt_df.raceId == 1034]
# race_1034_df["seconds"] = race_1034_df.milliseconds / 1000
# race_1034_df = race_1034_df.drop(columns = "milliseconds")
# race_1034_df

/Users/adelineshin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,raceId,driverId,lap,seconds
474870,1033,1,1,95.670
474871,1033,1,2,91.458
474872,1033,1,3,94.916
474873,1033,1,4,113.821
474874,1033,1,5,85.898
...,...,...,...,...
477087,1034,849,48,90.871
477088,1034,849,49,90.915
477089,1034,849,50,91.119
477090,1034,849,51,91.227


## Race Table Function

In [8]:
import pandas as pd

# Import all the data
drivers_df = pd.read_csv("./f1db_csv/drivers.csv").drop(columns = "url")
lap_times_df = pd.read_csv("./f1db_csv/lap_times.csv")
results_df = pd.read_csv("./f1db_csv/results.csv")
constructors_df = pd.read_csv("./f1db_csv/constructors.csv")
races_df = pd.read_csv("./f1db_csv/races.csv")

# Clean some names and create new variables
# drivers_df
drivers_df["number"] = drivers_df["number"].replace({r"\N": None})
drivers_df["driverName"] = drivers_df["forename"].str.cat(drivers_df["surname"],sep = " ")
drivers_df = drivers_df.drop(columns = ["forename", "surname"])

# lap_times_df
clean_lt_df = lap_times_df[["raceId", "driverId", "lap", "milliseconds"]]
clean_lt_df["seconds"] = clean_lt_df.milliseconds / 1000
clean_lt_df = clean_lt_df.drop(columns = "milliseconds")

# results_df
results_df["position"] = results_df["position"].replace({r"\N": 99})
results_df["position"] = results_df["position"].astype(int)

def create_race_table(year, race_name):
    races_temp = races_df[races_df.year == year]
    race_id = int(races_temp.raceId[races_temp.name == race_name])
    lap_times_1 = clean_lt_df[clean_lt_df.raceId == race_id]
    results_1 = results_df[results_df.raceId == race_id]
    df_1 = pd.merge(drivers_df[["driverId", "driverName", "number"]], lap_times_1, on = "driverId")
    df_2 = pd.merge(df_1, results_1[["resultId", "driverId", "constructorId", "position"]], on = "driverId")
    df_3 = pd.merge(df_2, constructors_df[["constructorId", "constructorRef"]], on = "constructorId")
    df_3["constructorRef"] = df_3["constructorRef"].str.title()
    df_4 = pd.merge(df_3, races_df[["raceId", "year", "name"]], on = "raceId")
    df_4 = df_4.sort_values(by = ["position", "lap"])
    return df_4

df_4 = create_race_table(2020, "British Grand Prix")
df_4

/Users/adelineshin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,driverId,driverName,number,raceId,lap,seconds,resultId,constructorId,position,constructorRef,year,name
0,1,Lewis Hamilton,44,1034,1,94.010,24686,131,1,Mercedes,2020,British Grand Prix
1,1,Lewis Hamilton,44,1034,2,126.445,24686,131,1,Mercedes,2020,British Grand Prix
2,1,Lewis Hamilton,44,1034,3,151.201,24686,131,1,Mercedes,2020,British Grand Prix
3,1,Lewis Hamilton,44,1034,4,152.874,24686,131,1,Mercedes,2020,British Grand Prix
4,1,Lewis Hamilton,44,1034,5,138.933,24686,131,1,Mercedes,2020,British Grand Prix
...,...,...,...,...,...,...,...,...,...,...,...,...
422,826,Daniil Kvyat,26,1034,7,94.402,24703,213,99,Alphatauri,2020,British Grand Prix
423,826,Daniil Kvyat,26,1034,8,93.521,24703,213,99,Alphatauri,2020,British Grand Prix
424,826,Daniil Kvyat,26,1034,9,93.384,24703,213,99,Alphatauri,2020,British Grand Prix
425,826,Daniil Kvyat,26,1034,10,93.461,24703,213,99,Alphatauri,2020,British Grand Prix


## Driver Race Reference Table

In [ ]:
driver_race_df

In [8]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Import all the data
drivers_df = pd.read_csv("./f1db_csv/drivers.csv").drop(columns = "url")
lap_times_df = pd.read_csv("./f1db_csv/lap_times.csv")
results_df = pd.read_csv("./f1db_csv/results.csv")
constructors_df = pd.read_csv("./f1db_csv/constructors.csv")
races_df = pd.read_csv("./f1db_csv/races.csv")

# Clean some names and create new variables
# drivers_df
drivers_df["number"] = drivers_df["number"].replace({r"\N": None})
drivers_df["driverName"] = drivers_df["forename"].str.cat(drivers_df["surname"],sep = " ")
drivers_df = drivers_df.drop(columns = ["forename", "surname"])

# Create the table of drivers and teams for each race
driver_race_df_1 = pd.merge(results_df[["raceId", "driverId", "constructorId"]], drivers_df[["driverName", "driverRef", "driverId"]], on = ["driverId"])
driver_race_df_2 = pd.merge(driver_race_df_1, constructors_df[["constructorRef", "constructorId", "name"]], on = "constructorId")
driver_race_df_2 = driver_race_df_2.rename({'name': 'constructorName'}, axis=1)
driver_race_df = pd.merge(driver_race_df_2, races_df[["year", "name", "raceId"]], on = "raceId")
driver_race_df = driver_race_df.sort_values(by = "year", ascending = False)

# Add color to every driver for each race
driver_race_df["color"] = [0] * len(driver_race_df.raceId)
for i in range(len(driver_race_df.raceId)):
    if driver_race_df.year[i] == 2020:
        if driver_race_df.constructorName[i] == "Mercedes":
            driver_race_df["color"][i] = "#00D2BE"
            i += 1
        elif driver_race_df.constructorName[i] == "Red Bull":
            driver_race_df["color"][i] = "#0600EF"
            i += 1
        elif driver_race_df.constructorName[i] == "Ferrari":
            driver_race_df["color"][i] = "#C00000"
            i += 1 
        elif driver_race_df.constructorName[i] == "Renault":
            driver_race_df["color"][i] = "#FFF500"
            i += 1 
        elif driver_race_df.constructorName[i] == "Haas F1 Team":
            driver_race_df["color"][i] = "#787878"
            i += 1     
        elif driver_race_df.constructorName[i] == "Racing Point":
            driver_race_df["color"][i] = "#F596C8"
            i += 1 
        elif driver_race_df.constructorName[i] == "AlphaTauri":
            driver_race_df["color"][i] = "#C8C8C8"
            i += 1 
        elif driver_race_df.constructorName[i] == "McLaren":
            driver_race_df["color"][i] = "#FF8700"
            i += 1 
        elif driver_race_df.constructorName[i] == "Alfa Romeo":
            driver_race_df["color"][i] = "#960000"
            i += 1 
        elif driver_race_df.constructorName[i] == "Williams":
            driver_race_df["color"][i] = "#0082FA"
            i += 1
    elif driver_race_df.year[i] == 2019:
        if driver_race_df.constructorName[i] == "Mercedes":
            driver_race_df.color[i] = "#00D2BE"
            i += 1
        elif driver_race_df.constructorName[i] == "Red Bull":
            driver_race_df.color[i] = "#1E41FF"
            i += 1
        elif driver_race_df.constructorName[i] == "Ferrari":
            driver_race_df.color[i] = "#DC0000"
            i += 1 
        elif driver_race_df.constructorName[i] == "Renault":
            driver_race_df.color[i] = "#FFF500"
            i += 1 
        elif driver_race_df.constructorName[i] == "Haas":
            driver_race_df.color[i] = "#F0D787"
            i += 1     
        elif driver_race_df.constructorName[i] == "Racing Point":
            driver_race_df.color[i] = "#F596C8"
            i += 1 
        elif driver_race_df.constructorName[i] == "Toro Rosso":
            driver_race_df.color[i] = "#469BFF"
            i += 1 
        elif driver_race_df.constructorName[i] == "Mclaren":
            driver_race_df.color[i] = "#FF8700"
            i += 1 
        elif driver_race_df.constructorName[i] == "Alfa Romeo":
            driver_race_df.color[i] = "#9B0000"
            i += 1 
        elif driver_race_df.constructorName[i] == "Williams":
            driver_race_df.color[i] = "#FFFFFF"
            i += 1
    elif driver_race_df.year[i] == 2018:
        if driver_race_df.constructorName[i] == "Mercedes":
            driver_race_df.color[i] = "#00D2BE"
            i += 1
        elif driver_race_df.constructorName[i] == "Red Bull":
            driver_race_df.color[i] = "#00327D"
            i += 1
        elif driver_race_df.constructorName[i] == "Ferrari":
            driver_race_df.color[i] = "#DC0000"
            i += 1 
        elif driver_race_df.constructorName[i] == "Renault":
            driver_race_df.color[i] = "#FFF500"
            i += 1 
        elif driver_race_df.constructorName[i] == "Haas":
            driver_race_df.color[i] = "#5A5A5A"
            i += 1     
        elif driver_race_df.constructorName[i] == "Force India":
            driver_race_df.color[i] = "#F596C8"
            i += 1 
        elif driver_race_df.constructorName[i] == "Toro Rosso":
            driver_race_df.color[i] = "#0032FF"
            i += 1 
        elif driver_race_df.constructorName[i] == "Mclaren":
            driver_race_df.color[i] = "#FF8700"
            i += 1 
        elif driver_race_df.constructorName[i] == "Sauber":
            driver_race_df.color[i] = "#9B0000"
            i += 1 
        elif driver_race_df.constructorName[i] == "Williams":
            driver_race_df.color[i] = "#FFFFFF"
            i += 1
    elif driver_race_df.year[i] == 2017:
        if driver_race_df.constructorName[i] == "Mercedes":
            driver_race_df.color[i] = "#00CFBA"
            i += 1
        elif driver_race_df.constructorName[i] == "Red Bull":
            driver_race_df.color[i] = "#00007D"
            i += 1
        elif driver_race_df.constructorName[i] == "Ferrari":
            driver_race_df.color[i] = "#C30000"
            i += 1 
        elif driver_race_df.constructorName[i] == "Renault":
            driver_race_df.color[i] = "#FFD800"
            i += 1 
        elif driver_race_df.constructorName[i] == "Haas":
            driver_race_df.color[i] = "#6C0000"
            i += 1     
        elif driver_race_df.constructorName[i] == "Force India":
            driver_race_df.color[i] = "#FF80C7"
            i += 1 
        elif driver_race_df.constructorName[i] == "Toro Rosso":
            driver_race_df.color[i] = "#0000FF"
            i += 1 
        elif driver_race_df.constructorName[i] == "Mclaren":
            driver_race_df.color[i] = "#FF7B08"
            i += 1 
        elif driver_race_df.constructorName[i] == "Sauber":
            driver_race_df.color[i] = "#006EFF"
            i += 1 
        elif driver_race_df.constructorName[i] == "Williams":
            driver_race_df.color[i] = "#FFFFFF"
            i += 1
    else:
        number_of_colors = 12
        color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
        

driver_race_df.head(40)
driver_race_df.to_csv("./colors.csv")

In [16]:
import random
constructors_df["color"] = [0] * len(constructors_df.name)
for i in range(len(constructors_df.name)):
    if constructors_df.name[i] == "Mercedes":
        constructors_df["color"][i] = "#00D2BE"
    elif constructors_df.name[i] == "Red Bull":
        constructors_df["color"][i] = "#0600EF"
    elif constructors_df.name[i] == "Ferrari":
        constructors_df["color"][i] = "#C00000"
    elif constructors_df.name[i] == "Renault":
        constructors_df["color"][i] = "#FFF500"
    elif constructors_df.name[i] == "Haas F1 Team":
        constructors_df["color"][i] = "#787878"  
    elif constructors_df.name[i] == "Racing Point":
        constructors_df["color"][i] = "#F596C8"
    elif constructors_df.name[i] == "AlphaTauri":
        constructors_df["color"][i] = "#C8C8C8"
    elif constructors_df.name[i] == "McLaren":
        constructors_df["color"][i] = "#FF8700"
    elif constructors_df.name[i] == "Alfa Romeo":
        constructors_df["color"][i] = "#960000"
    elif constructors_df.name[i] == "Williams":
        constructors_df["color"][i] = "#0082FA"
    elif constructors_df.name[i] == "Toro Rosso":
        constructors_df["color"][i] = "#469BFF"
    elif constructors_df.name[i] == "BMW Sauber":
        constructors_df["color"][i] = "#006EFF"
    else:
        colors = ["red", "green", "blue", "orange", "purple", "pink", "yellow", "white", "bisque", "turquoise",
                 "forestgreen", "darkgreen", "cornflowerblue", "silver", "crimson", "fuschia"]
        constructors_df["color"][i] = random.choice(colors)

constructors_df.to_csv("./f1db_csv/constructors_colors.csv")

/Users/adelineshin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/adelineshin/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/adelineshin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [29]:
constructor_colors_df = pd.read_csv("./f1db_csv/constructors_colors.csv")
races_temp = races_df[races_df.year == 2020]
race_id = int(races_temp.raceId[races_temp.name == "British Grand Prix"])
lap_times_1 = clean_lt_df[clean_lt_df.raceId == race_id]
results_1 = results_df[results_df.raceId == race_id]
df_1 = pd.merge(drivers_df[["driverId", "driverName", "number"]], lap_times_1, on = "driverId")
df_2 = pd.merge(df_1, results_1[["resultId", "driverId", "constructorId", "position"]], on = "driverId")
df_3 = pd.merge(df_2, constructors_df[["constructorId", "constructorRef"]], on = "constructorId")
df_3["constructorRef"] = df_3["constructorRef"].str.title()
df_4 = pd.merge(df_3, races_df[["raceId", "year", "name"]], on = "raceId")
df_4 = df_4.sort_values(by = ["position", "lap"])
colored_df = pd.merge(df_4, constructor_colors_df[["constructorId", "color"]], on = "constructorId")
color_palette = pd.Series(colored_df.color.values, index = colored_df.driverName).to_dict()

colored_df

,driverId,driverName,number,raceId,lap,seconds,resultId,constructorId,position,constructorRef,year,name,color
0,1,Lewis Hamilton,44,1034,1,94.010,24686,131,1,Mercedes,2020,British Grand Prix,#00D2BE
1,1,Lewis Hamilton,44,1034,2,126.445,24686,131,1,Mercedes,2020,British Grand Prix,#00D2BE
2,1,Lewis Hamilton,44,1034,3,151.201,24686,131,1,Mercedes,2020,British Grand Prix,#00D2BE
3,1,Lewis Hamilton,44,1034,4,152.874,24686,131,1,Mercedes,2020,British Grand Prix,#00D2BE
4,1,Lewis Hamilton,44,1034,5,138.933,24686,131,1,Mercedes,2020,British Grand Prix,#00D2BE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,154,Romain Grosjean,8,1034,49,89.782,24701,210,16,Haas,2020,British Grand Prix,#787878
891,154,Romain Grosjean,8,1034,50,90.013,24701,210,16,Haas,2020,British Grand Prix,#787878
892,154,Romain Grosjean,8,1034,51,90.318,24701,210,16,Haas,2020,British Grand Prix,#787878
893,154,Romain Grosjean,8,1034,52,91.686,24701,210,16,Haas,2020,British Grand Prix,#787878


In [6]:
drivers = ["ver", "ham", "vet"]
times = [1, 5, 2]

df = pd.DataFrame({"drivers" : drivers, "times" : times})
df["lap_min"] = min(times)
df

,drivers,times,lap_min
0,ver,1,1
1,ham,5,1
2,vet,2,1
